# dLNk GPT Uncensored - AutoTrain on GPU

This notebook trains the dLNk GPT uncensored model using Hugging Face AutoTrain with GPU acceleration.

**Requirements:**
- GPU Runtime (T4, A100, or V100)
- Hugging Face Token
- 12-16 hours training time

**Steps:**
1. Enable GPU: Runtime → Change runtime type → GPU
2. Run all cells in order
3. Monitor training progress

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required packages with compatible versions
!pip install -q --upgrade huggingface_hub
!pip install -q transformers accelerate bitsandbytes peft datasets
!pip install -q autotrain-advanced

## 2. Login to Hugging Face

In [ ]:
from huggingface_hub import login

# Enter your Hugging Face token
HF_TOKEN = ""  # Replace with your token

login(token=HF_TOKEN)
print("Logged in to Hugging Face")

## 3. Load Dataset

In [ ]:
from datasets import load_dataset

# Load dataset from Hugging Face Hub
dataset = load_dataset("dlnkgpt/dlnkgpt-uncensored-dataset")

print(f"Dataset loaded:")
print(f"  Train: {len(dataset['train']):,} examples")
print(f"  Validation: {len(dataset['validation']):,} examples")

# Show sample
print(f"\nSample:")
print(dataset['train'][0]['text'][:200] + "...")

## 4. Prepare Dataset for Training

In [ ]:
# Save dataset locally
dataset.save_to_disk("./autotrain_data")
print("Dataset saved locally")

## 5. Configure Training Parameters

In [ ]:
import os

# Set environment variable
os.environ['HF_TOKEN'] = HF_TOKEN

# Training configuration
PROJECT_NAME = "dlnkgpt-uncensored"
MODEL_NAME = "EleutherAI/gpt-j-6b"
DATA_PATH = "./autotrain_data"
TEXT_COLUMN = "text"

# Training parameters
EPOCHS = 3
BATCH_SIZE = 4
LEARNING_RATE = 2e-5
WARMUP_RATIO = 0.1
GRADIENT_ACCUMULATION = 8
BLOCK_SIZE = 512

# LoRA parameters
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

print("Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Effective Batch Size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  LoRA Enabled: True")

## 6. Start Training

**Note:** This will take 12-16 hours on T4 GPU. You can close the browser and come back later.

In [ ]:
# Launch AutoTrain
!autotrain llm \
  --train \
  --project-name {PROJECT_NAME} \
  --model {MODEL_NAME} \
  --data-path {DATA_PATH} \
  --text-column {TEXT_COLUMN} \
  --lr {LEARNING_RATE} \
  --epochs {EPOCHS} \
  --batch-size {BATCH_SIZE} \
  --warmup-ratio {WARMUP_RATIO} \
  --gradient-accumulation {GRADIENT_ACCUMULATION} \
  --block_size {BLOCK_SIZE} \
  --logging-steps 100 \
  --eval-strategy steps \
  --save-total-limit 2 \
  --peft \
  --lora-r {LORA_R} \
  --lora-alpha {LORA_ALPHA} \
  --lora-dropout {LORA_DROPOUT} \
  --mixed-precision fp16 \
  --push-to-hub \
  --username dlnkgpt \
  --token $HF_TOKEN

## 7. Test Trained Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Load base model
print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/gpt-j-6b",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter
print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(
    base_model,
    "dlnkgpt/dlnkgpt-uncensored"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("dlnkgpt/dlnkgpt-uncensored")

print("Model loaded successfully")

# Test generation
def generate_response(prompt, max_length=200):
    formatted_prompt = f"### Instruction:\n{prompt}\n\n### Response:\n"
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=0.7,
        do_sample=True,
        top_p=0.9
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test prompts
test_prompts = [
    "Explain artificial intelligence",
    "What is machine learning?",
    "Describe neural networks"
]

print("\nTesting model...")
for i, prompt in enumerate(test_prompts, 1):
    print(f"\nTest {i}: {prompt}")
    response = generate_response(prompt)
    print(f"Response: {response}")
    print("-" * 70)

## Summary

**Training Complete!**

Your model has been:
- Trained on 54,000 examples
- Fine-tuned for 3 epochs
- Pushed to Hugging Face Hub
- Ready to use

**Model URL:** https://huggingface.co/dlnkgpt/dlnkgpt-uncensored

**Next Steps:**
1. Test the model with various prompts
2. Integrate with your API
3. Deploy to production